# ライブラリ

In [1]:
# !pip install librosa
# !pip install tqdm

In [ ]:
import librosa
import os
import matplotlib.pyplot as plt
import numpy as np
import IPython.display
import pandas as pd
import statistics as sta
import torch

print(librosa.__version__)
print(torch.__version__)


: 


# ファイル読み込み

In [ ]:
# # 変数定義
sr = 44100 #サンプリング周波数

# #ファイルパスの指定
# train = "../raw_data/contacting_object/train_data/" #NASのデータ
# # audio_dir = "../Data_contacting_object/initial/FOLDER02" #ローカルデータ
# eval = "../raw_data/contacting_object/initial/FOLDER03/" 


# #学習用データの読み込み
# audio_dir = train
# subFolders = [f for f in os.listdir(audio_dir) if os.path.isdir(os.path.join(audio_dir, f))]
# subFolders = sorted(subFolders) #昇順に並び替え

# #ファイル読み込み
# # originDatasに音声データを格納していく
# originDatas = []
# for subFolder in subFolders:
#     filePath = audio_dir+"/"+subFolder+"/"+subFolder+"_Tr1.WAV"
#     originData, sr = librosa.load(filePath, sr = sr)
#     originDatas.append(originData)

# #評価用データの読み込み
# folderPath = eval
# subFolders = [f for f in os.listdir(folderPath) if os.path.isdir(os.path.join(folderPath, f))]
# subFolders = sorted(subFolders) #昇順に並び替え

# #ファイル読み込み
# # originDatas_eに音声データを格納していく
# originDatas_e = []
# for subFolder in subFolders:
#     filePath = folderPath+"/"+subFolder+"/"+subFolder+"_Tr1.WAV"
#     originData_e, sr = librosa.load(filePath, sr = sr)
#     originDatas_e.append(originData_e)


# #1データから30試行をトリミングを22パターン分行い，soundDatasetに格納する
# # soundDatasetの構造：[パターン(10)][試行(50)][サンプリングデータ(66150)]

# flag_amp = 0.1 #各試行の合図を検知する基準振幅
# trimSkip = int(sr*0.4)
# trimTime = int(sr*1.5)#1試行あたりのデータ長
# dataNum = 50
# trial = 0
# soundDataset =([])

# while trial <len(originDatas):
#     index = 0
#     trimDatas = ([]) #1データ(30試行)分のトリミングデータのリストを初期化
#     originData = np.array(originDatas[trial])
#     while index <len(originData):
#         if originData[index] >= flag_amp:
#             trimData = np.array(originData[index+trimSkip:index+trimTime+trimSkip]) #trim_dataにそのindexからindex+trimTimeのデータを格納する
#             trimDatas = np.append(trimDatas ,trimData, axis=0)  #trimDatasに追加する
#             index += trimTime
#         else:
#             index +=1
#         if len(trimDatas) >=dataNum*trimTime:
#             break
#     soundDataset = np.append(soundDataset,np.array(trimDatas),axis=0)
#     trial += 1

# soundDataset = soundDataset.reshape(len(originDatas),dataNum,trimTime)

# print(soundDataset.shape)

# import pickle
# f = open('soundDataset.pickle','wb')
# pickle.dump(soundDataset,f)

: 

In [ ]:
import pickle
f = open('soundDataset.pickle','rb')
soundDataset = pickle.load(f)

: 

In [ ]:
#正解ラベルリストの作成

#object層
objectLabel = [
    "Y-shirt", 
    "Jeans", 
    "Sweatshirt", 

    "Blanket", 
    "bedquilt", 
    "Pillow", 
    
    "Thick Book", 
    "Thin Book", 
    "Cardboard", 
    
    "Mousepad", 
    "Chair", 
    "Sofa", 
    
    "Metal Desk", 
    "Laptop", 
    "AlumiRack", 
    
    "WoodDesk", 
    "WoodShelf", 
    "Floor", 
    
    "Small Organizer", 
    "PlaBag", 
    "PlaShelf"
]

#material層
matLabel = [
    "Clothing",
    "Clothing", 
    "Paper", 
    "Memory Foam", 
    "Metal", 
    "Wood", 
    "Plastic"
]

matNum = [0,0,1,2,3,4,5]

#soft-hard層
shLabel = [
    "soft",
    "hard",
]

shNum = [0,1]





: 

In [ ]:
print(len(soundDataset))

: 

### 評価データセット

In [ ]:
IPython.display.Audio(data=soundDataset[1][0], rate=sr)

: 

### スペクトログラムの描写

In [ ]:
# # パターンごとにスペクロログラムの描写
# for patern in range(len(soundDataset)):
#     trimData = soundDataset[patern,0]
#     spectrogram = librosa.feature.melspectrogram(y=trimData, sr=sr)# スペクトログラムを計算
#     librosa.display.specshow(librosa.power_to_db(spectrogram, ref=np.max)\
#                              , y_axis='mel', x_axis='time')# スペクトログラムを表示
    # plt.colorbar(format='%+2.0f dB')# カラーバーを追加
    # plt.ylim(0,500)
    # plt.title('Mel Spectrogram_'+objectLabel[patern])# グラフのタイトルを設定
    # plt.show()# グラフを表示


: 

: 

# 特徴量抽出

In [ ]:

for i in range(soundDataset.shape[0]):
    title = 'FFT_'+objectLabel[i]
    #FFT
    data = soundDataset[i][0] # checked data
    win_wid = 4096
    win_ham = np.blackman(win_wid)

    #window幅で割り切れない分を削除する
    if data.shape[0] % win_wid != 0:
        data = data[0:-(data.shape[0] % win_wid)]



    #overlap & split window 
    overlap_rate = 0.5
    overlap = int(win_wid*overlap_rate)
    step = win_wid - overlap
    frames = []
    for start in range(0, len(data),step):
        end = start + win_wid  # 'end'を定義
        if end > len(data):
            break
        frame = data[start:end]
        frames.append(frame)

    #FFT
    fft_ret = np.fft.rfft(frames * win_ham) #FFT
    fft_freq = np.fft.rfftfreq(win_wid, 1/sr) #周波数軸のデータ作成

    #transfer amp to log
    log_fft_ret = 20*np.log10(np.abs(fft_ret)) # 対数データの取得←データを見やすくするため


    #plot result
    result_fft = np.mean(log_fft_ret, axis = 0)
    plt.title(title)
    plt.plot(fft_freq,result_fft)
    plt.xlim(100,2000)
    plt.ylim(-40,40)
    # plt.savefig('data/figure_obj/'+title+'.pdf')
    # plt.show()


: 

In [ ]:
# 特徴量の次元数
num_feature = 40

# 空のmfccSetを初期化
mfccSet = np.empty((0, num_feature))

# ループを使用してMFCCを抽出してmfccSetに追加
for pattern in range(soundDataset.shape[0]):
    for trial in range(soundDataset.shape[1]):
        # MFCCを計算
        mfccs = librosa.feature.mfcc(y=soundDataset[pattern][trial], sr=sr)
        
        # MFCCの各次元の平均を算出
        mean = mfccs.mean(axis=1)
        
        # MFCCの各次元の標準偏差を算出
        std = np.std(mfccs, axis=1)
        
        # mean, max_val, min_val, std をまとめた配列を作成
        combined_stats = np.concatenate([mean,  std])

        # mfccSetに追加
        mfccSet = np.append(mfccSet, [combined_stats], axis=0)

# 形状を確認
print(mfccSet.shape)


: 

In [ ]:
# 形状を確認
print(mfccSet.shape)
print(mfccs.shape)

: 

# データセット作成

In [ ]:
import pandas as pd
# num_feature次元のMFCCのデータフレームを作成
Dataset = pd.DataFrame(data=mfccSet)


Dataset['objectNum'] = [i // soundDataset.shape[1] for i in range(mfccSet.shape[0])]
Dataset['matNum'] = np.repeat(matNum, 150)[:1050]
Dataset['shNum'] = np.repeat(shNum, 600)[:1050]

# データセットに'objectLabel'の列を追加
Dataset['object'] = np.repeat(objectLabel, 50)[:1050]

# # データセットに'matLabel'の列を追加
Dataset['mat'] = np.repeat(matLabel, 150)[:1050]

# # データセットに'SHLabel'の列を追加
Dataset['sh'] = np.repeat(shLabel, 600)[:1050]




# Display the DataFrame
Dataset

: 

In [ ]:
! ls ../

: 

In [ ]:
# trial1 = Dataset.iloc[Dataset.index // 10 % 10 <= 4]
# trial2 = Dataset.iloc[Dataset.index // 10 % 10 > 4]

: 

# ニューラルネット

In [ ]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを取得
jst = pytz.timezone('Asia/Tokyo')

# 現在の日本時間を取得
now = datetime.now(jst)

# 時刻を指定された形式の文字列に変換
time = "_"+now.strftime("%Y%m%d%H%M")

print(time)

: 

### 物体推定モデル

In [ ]:

audio_origin = soundDataset.reshape(1050, -1)

t_df = Dataset.objectNum


audio_tensor_dataset = torch.tensor(audio_origin, dtype=torch.float32)
label_tensor_sh = torch.tensor(t_df, dtype=torch.int64)

# x と t を組み合わせて TensorDataset を作成
dataset_origin = torch.utils.data.TensorDataset(audio_tensor_dataset, label_tensor_sh)
dataset_origin

# データセットを指定する
dataset = dataset_origin
print(len(dataset))
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%

n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train_origin, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

print(train_origin)


#学習データをnpへ変換
train_np = np.array([data.numpy() for data, _ in train_origin])
train_np_label = np.array([label for _, label in train_origin])

# #augmentation
train_aug_up = train_np*1.1
train_aug_down = train_np * 0.9
wn = np.random.randn(train_np.shape[0],66150)
train_aug_wn = train_np + wn
# print(train_np)
# print(train_aug_up)


train_np_aug = np.concatenate([train_aug_up,train_aug_down])
label_all = np.concatenate([train_np_label,train_np_label])
print(len(train_np_aug))
print(label_all)

train_np_aug = torch.tensor(train_np_aug,dtype=torch.float32)
label_all = torch.tensor(label_all, dtype=torch.int64)

train_aug = torch.utils.data.TensorDataset(train_np_aug, label_all)



train = train_origin

print(len(train))

# バッチサイズ
batch_size = 50

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)



: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class objNet(nn.Module):
    def __init__(self):
        super(objNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 40, 8),
            nn.BatchNorm1d(40),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(40, 40, 8),
            nn.BatchNorm1d(40),
            nn.ReLU()
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(40, 21, 8),
            nn.BatchNorm1d(21),
            nn.ReLU()
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(21, 21, 8),
            nn.BatchNorm1d(21),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(42, 21),  # 入力サイズを修正
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(21, 21),  # 出力サイズを修正
            nn.ReLU(),
            nn.Dropout(0.25),

            nn.Linear(21,1),
            # nn.Dropout(0.25)

        )

    def forward(self, x):
        h = self.conv1(x)
        h = self.conv2(h)
        h = F.max_pool1d(h, kernel_size=160)
          
        h = self.conv3(h)
        h = F.max_pool1d(h, kernel_size=3) 
        h = self.conv4(h) 
        h = F.max_pool1d(h, kernel_size=3) 
        h = self.classifier(h)
        h = h.view(h.size(0), -1)  # Flatten
        return h

# モデルのインスタンス化
objnet = objNet()

# モデルの出力を計算してみる
input_data = torch.randn(10,1,66150)
print(input_data.shape)  
output = objnet(input_data)
print("Output shape:", output.shape)


: 

In [ ]:
# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

: 

In [ ]:
torch.cuda.current_device()

: 

In [ ]:
# エポックの数
max_epoch = 50
     
# モデルの初期化
torch.manual_seed(0)



# モデルのインスタンス化とデバイスへの転送
net = objNet().to(device)

criterion = F.cross_entropy
criterion     

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net.train()
    for batch in train_loader:
        x, t = batch
        x = torch.unsqueeze(x, 1)
        # t = torch.unsqueeze(t, 0)
        x = x.to(device)
        t = t.to(device)
        
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
            x = torch.unsqueeze(x, 1)
            # t = torch.unsqueeze(t, 0)
            x = x.to(device)
            t = t.to(device)
            
            y = net(x)
            loss = criterion(y, t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')




: 

In [ ]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを取得
jst = pytz.timezone('Asia/Tokyo')

# 現在の日本時間を取得
now = datetime.now(jst)

# 時刻を指定された形式の文字列に変換
time = "_"+now.strftime("%Y%m%d%H%M")

# 学習曲線の可視化

title = 'Training and Validation Losses object'
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.title(title)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0,5)
plt.legend()
# plt.savefig("data/"+title+time+".pdf")

plt.show()

: 

In [ ]:
# # y_labelとtを同じデバイスに移動
# y_label = y_label.to(y.device)
# t = t.to(y.device)

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y, dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc

# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)


: 

### 材質推定モデル


In [ ]:
# モデルの初期化
torch.manual_seed(0)


audio_origin = soundDataset.reshape(1050, -1)
print(soundDataset.shape)
print(audio_origin.shape)

t_df = Dataset.matNum
audio_tensor_dataset = torch.tensor(audio_origin, dtype=torch.float32)
label_tensor = torch.tensor(t_df, dtype=torch.int64)

# x と t を組み合わせて TensorDataset を作成
dataset_origin = torch.utils.data.TensorDataset(audio_tensor_dataset, label_tensor)
dataset_origin

dataset_augu = augumentation(audio_origin,66150,Dataset.matNum)

# データセットを指定する
dataset = dataset_origin
print(len(dataset))
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%

n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

train = train 
print(len(train))

# バッチサイズ
batch_size = 50

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)
len(train_loader)

: 

In [ ]:

audio_origin = soundDataset.reshape(1050, -1)

t_df = Dataset.matNum


audio_tensor_dataset = torch.tensor(audio_origin, dtype=torch.float32)
label_tensor = torch.tensor(t_df, dtype=torch.int64)

# x と t を組み合わせて TensorDataset を作成
dataset_origin = torch.utils.data.TensorDataset(audio_tensor_dataset, label_tensor)
dataset_origin

# データセットを指定する
dataset = dataset_origin
print(len(dataset))
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%

n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train_origin, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

print(train_origin)


#学習データをnpへ変換
train_np = np.array([data.numpy() for data, _ in train_origin])
train_np_label = np.array([label for _, label in train_origin])

# #augmentation
train_aug_up = train_np*1.2
train_aug_down = train_np * 0.8
wn = np.random.randn(train_np.shape[0],66150)
train_aug_wn = train_np + wn
print(train_aug_up.shape)


train_np_aug = np.concatenate([train_aug_up,train_aug_down,train_aug_wn])
label_all = np.concatenate([train_np_label,train_np_label,train_np_label])
print(len(train_np_aug))
print(label_all)

train_np_aug = torch.tensor(train_np_aug,dtype=torch.float32)
label_all = torch.tensor(label_all, dtype=torch.int64)

train_aug = torch.utils.data.TensorDataset(train_np_aug, label_all)



train = train_origin

print(len(train))

# バッチサイズ
batch_size = 50

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)



: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class matNet(nn.Module):
    def __init__(self):
        super(matNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 40, 8),
            nn.BatchNorm1d(40),
            nn.ReLU(),
            # nn.Dropout(0.25),
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(40, 40, 8),
            nn.BatchNorm1d(40),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(40, 6, 8),
            nn.BatchNorm1d(6),
            nn.ReLU()
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(6, 6, 8),
            nn.BatchNorm1d(6),
            nn.ReLU()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(42, 21),  # 入力サイズを修正
            nn.ReLU(),
            # nn.Dropout(0.25),
            
            nn.Linear(21, 10),  # 出力サイズを修正
            nn.ReLU(),
            # nn.Dropout(0.25),

            nn.Linear(10, 10),  # 出力サイズを修正
        )

    def forward(self, x):
        h = self.conv1(x)
        h = self.conv2(h)
        h = F.max_pool1d(h, kernel_size=160)
          
        h = self.conv3(h)
        h = F.max_pool1d(h, kernel_size=3) 
        h = self.conv4(h) 
        h = F.max_pool1d(h, kernel_size=3) 
        h = self.classifier(h)
        h = h.view(h.size(0), -1)  # Flatten
        return h

# モデルのインスタンス化
matnet = matNet()

# モデルの出力を計算してみる
input_data = torch.randn(10,1,66150)
print(input_data.shape)  
output = matnet(input_data)
print("Output shape:", output.shape)

: 

In [ ]:
# エポックの数
max_epoch = 100
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

# モデルのインスタンス化とデバイスへの転送
net = matNet().to(device)

criterion = F.cross_entropy
criterion     

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net.train()
    for batch in train_loader:
        x, t = batch
        x = torch.unsqueeze(x, 1)
        # t = torch.unsqueeze(t, 0)
        x = x.to(device)
        t = t.to(device)
        
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
            x = torch.unsqueeze(x, 1)
            # t = torch.unsqueeze(t, 0)
            x = x.to(device)
            t = t.to(device)
            
            y = net(x)
            loss = criterion(y, t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')




: 

In [ ]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを取得
jst = pytz.timezone('Asia/Tokyo')

# 現在の日本時間を取得
now = datetime.now(jst)

# 時刻を指定された形式の文字列に変換
time = "_"+now.strftime("%Y%m%d%H%M")

# 学習曲線の可視化

title = 'Training and Validation Losses material'
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.title(title)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0,5)
plt.legend()
plt.savefig("data/"+title+time+".pdf")

plt.show()

: 

In [ ]:
# y_labelとtを同じデバイスに移動
y_label = y_label.to(y.device)
t = t.to(y.device)

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y, dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)


: 

In [ ]:
device = torch.cuda.is_available() 
print(device)

import torch
print(torch.__version__)
print(torch.version.cuda)


: 

### 剛柔推定モデル

In [ ]:
# モデルの初期化
torch.manual_seed(0)


audio_origin = soundDataset.reshape(1050, -1)
print(soundDataset.shape)
print(audio_origin.shape)

t_df = Dataset.shNum
audio_tensor_dataset = torch.tensor(audio_origin, dtype=torch.float32)
label_tensor_sh = torch.tensor(t_df, dtype=torch.int64)

# x と t を組み合わせて TensorDataset を作成
dataset_origin = torch.utils.data.TensorDataset(audio_tensor_dataset, label_tensor_sh)
dataset_origin

# dataset_augu = augumentation(audio_origin,66150,Dataset.shNum)

# データセットを指定する
dataset = dataset_origin
print(len(dataset))
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%

n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

train = train 
print(len(train))

# バッチサイズ
batch_size = 50

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)
len(train_loader)

: 

In [ ]:

audio_origin = soundDataset.reshape(1050, -1) # type: ignore

t_df = Dataset.shNum


audio_tensor_dataset = torch.tensor(audio_origin, dtype=torch.float32)
label_tensor = torch.tensor(t_df, dtype=torch.int64)

# x と t を組み合わせて TensorDataset を作成
dataset_origin = torch.utils.data.TensorDataset(audio_tensor_dataset, label_tensor)
dataset_origin

# データセットを指定する
dataset = dataset_origin
print(len(dataset))
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%

n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train_origin, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

print(train_origin)


#学習データをnpへ変換
train_np = np.array([data.numpy() for data, _ in train_origin])
train_np_label = np.array([label for _, label in train_origin])

# #augmentation
train_aug_up = train_np*1.2
train_aug_down = train_np * 0.8
wn = np.random.randn(train_np.shape[0],66150)
train_aug_wn = train_np + wn
print(train_aug_up.shape)


train_np_aug = np.concatenate([train_aug_up,train_aug_down,train_aug_wn])
label_all = np.concatenate([train_np_label,train_np_label,train_np_label])
print(len(train_np_aug))
print(label_all)

train_np_aug = torch.tensor(train_np_aug,dtype=torch.float32)
label_all = torch.tensor(label_all, dtype=torch.int64)

train_aug = torch.utils.data.TensorDataset(train_np_aug, label_all)



train = train_origin 

print(len(train))

# バッチサイズ
batch_size = 50

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)



: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class shNet(nn.Module):
    def __init__(self):
        super(shNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 40, 8),
            nn.BatchNorm1d(40),
            nn.ReLU(),
            # nn.Dropout(0.25),
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(40, 40, 8),
            nn.BatchNorm1d(40),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(40, 2, 8),
            nn.BatchNorm1d(2),
            nn.ReLU()
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(2, 2, 8),
            nn.BatchNorm1d(2),
            nn.ReLU()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(42, 21),  # 入力サイズを修正
            nn.ReLU(),
            # nn.Dropout(0.25),
            
            nn.Linear(21, 2),  # 出力サイズを修正
            nn.ReLU(),
            # nn.Dropout(0.25),

            nn.Linear(2,2),  # 出力サイズを修正
        )

    def forward(self, x):
        h = self.conv1(x)
        h = self.conv2(h)
        h = F.max_pool1d(h, kernel_size=160)
          
        h = self.conv3(h)
        h = F.max_pool1d(h, kernel_size=3) 
        h = self.conv4(h) 
        h = F.max_pool1d(h, kernel_size=3) 
        h = self.classifier(h)
        h = h.view(h.size(0), -1)  # Flatten
        return h

# モデルのインスタンス化
shnet = shNet()

# モデルの出力を計算してみる
input_data = torch.randn(10,1,66150)
print(input_data.shape)  
output = shnet(input_data)
print("Output shape:", output.shape)

: 

In [ ]:
# エポックの数
max_epoch = 100
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# モデルのインスタンス化とデバイスへの転送
net = shNet().to(device)

criterion = F.cross_entropy
# criterion     

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net.train()
    for batch in train_loader:
        x, t = batch
        x = torch.unsqueeze(x, 1)
        # t = torch.unsqueeze(t, 0)
        x = x.to(device)
        t = t.to(device)
        
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
            x = torch.unsqueeze(x, 1)
            # t = torch.unsqueeze(t, 0)
            x = x.to(device)
            t = t.to(device)
            
            y = net(x)
            loss = criterion(y, t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')




: 

In [ ]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを取得
jst = pytz.timezone('Asia/Tokyo')

# 現在の日本時間を取得
now = datetime.now(jst)

# 時刻を指定された形式の文字列に変換
time = "_"+now.strftime("%Y%m%d%H%M")

# 学習曲線の可視化

title = 'Training and Validation Losses Soft-hard'
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.title(title)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0,5)
plt.legend()
# plt.savefig("data/"+title+time+".pdf")

plt.show()

: 

In [ ]:


# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y, dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)


: 

# 転移学習


In [ ]:
# モデルの初期化
torch.manual_seed(0)


audio_origin = soundDataset.reshape(1050, -1)
print(soundDataset.shape)
print(audio_origin.shape)

t_df = Dataset.matNum
audio_tensor_dataset = torch.tensor(audio_origin, dtype=torch.float32)
label_tensor = torch.tensor(t_df, dtype=torch.int64)

# x と t を組み合わせて TensorDataset を作成
dataset_origin = torch.utils.data.TensorDataset(audio_tensor_dataset, label_tensor)
dataset_origin

# データセットを指定する
dataset = dataset_origin
print(len(dataset))
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%

n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

train = train 
print(len(train))

# バッチサイズ
batch_size = 50

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)
len(train_loader)

: 

In [ ]:
# 転移学習
import copy
import torch.nn as nn

# 前提としてshNetが定義されている必要があります
# print(net)

transNet = shNet()

# パラメータ固定
for param in transNet.parameters():
    param.requires_grad = False

print(transNet.classifier)

# 畳み込み層の出力を計算するための入力サイズ（例：入力の長さが42の場合）
input_length = 42
conv_kernel_size = 8
conv_output_size = input_length - conv_kernel_size + 1

transNet.classifier = nn.Sequential(
    nn.Conv1d(2, 6, conv_kernel_size),
    nn.BatchNorm1d(6),
    nn.ReLU(),

    nn.Flatten(),  # 畳み込み層の出力をフラット化する

    nn.Linear(6 * conv_output_size, 21),  # 入力サイズを修正
    nn.ReLU(),
    
    nn.Linear(21, 10),  # 出力サイズを修正
    nn.ReLU(),

    nn.Linear(10, 10),  # 出力サイズを修正
)

print(transNet)


: 

In [ ]:
# エポックの数
max_epoch = 100
     
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# モデルのインスタンス化とデバイスへの転送
net = transNet

net = net.to(device)
# net = trans_net

criterion = F.cross_entropy
  

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net.train()
    for batch in train_loader:
        x, t = batch
        x = torch.unsqueeze(x, 1)
        # t = torch.unsqueeze(t, 0)
        x = x.to(device)
        t = t.to(device)
        
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
            x = torch.unsqueeze(x, 1)
            # t = torch.unsqueeze(t, 0)
            x = x.to(device)
            t = t.to(device)
            
            y = net(x)
            loss = criterion(y, t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')




: 

In [ ]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを取得
jst = pytz.timezone('Asia/Tokyo')

# 現在の日本時間を取得
now = datetime.now(jst)

# 時刻を指定された形式の文字列に変換
time = "_"+now.strftime("%Y%m%d%H%M")

# 学習曲線の可視化

title = 'Training and Validation Losses Soft-hard'
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.title(title)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0,5)
plt.legend()
# plt.savefig("data/"+title+time+".pdf")

plt.show()

: 

In [ ]:


# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y, dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)


: 

# EfficientATのファインチューニング

### torchデータセットの作成

In [ ]:
# パターンごとにスペクロログラムの保存
df = []
trial_num = 50
for patern in range (len(soundDataset)):
    for trial in range(trial_num):
        trimData = soundDataset[patern,trial]
        spectrogram = librosa.feature.melspectrogram(y=trimData, sr=sr)# スペクトログラムを計算
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)    
        df.append(spectrogram_db)

df = np.array(df)
print(df.shape)
print(type(df))

: 

In [ ]:
print(df.shape) #(1050,128,130)


: 

### soft-hard 1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

: 

In [ ]:

# ラベルの指定
t_df = Dataset.shNum

# テンソル形式に変換
df_tenosor = torch.tensor(df, dtype=torch.float32)
df_tenosor = torch.unsqueeze(df_tenosor, 1)
label_tensor = torch.tensor(t_df, dtype=torch.int64)
print(df_tenosor.size())

# x と t を組み合わせて TensorDataset を作成
dataset = torch.utils.data.TensorDataset(df_tenosor, label_tensor)

# 各データセットのサンプル数を決定
# train : val: test = 50%　: 25% : 25%
n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# バッチサイズ
batch_size = 25

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

print(train_loader)

for batch in train_loader:
    print(batch)

: 

: 

In [ ]:
# modelのインポート
from models.dymn.model import get_model as get_dymn
model = get_dymn(pretrained_name="dymn10_as")

#パラメータの更新を許可
for param in model.parameters():
    param.requires_gred = True


# print(model.classifier)

# 出力層の最後だけ変更
# model.classifier[5] = torch.nn.Linear(1280,2)
model.classifier= nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=960, out_features=1280, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=527, bias=True),
    nn.Linear(in_features=527, out_features=176, bias=True),  # 新しい層
    nn.Linear(in_features=176, out_features=2, bias=True)  # 新しい層

)
print(model)

: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# エポックの数
max_epoch = 20
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# モデルのインスタンス化とデバイスへの転送
net1 = model.cuda()

criterion = F.cross_entropy
  
# 最適化手法の選択
optimizer = torch.optim.SGD(net1.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net1.train()
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net1(x)
        loss = criterion(y[0], t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net1.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
  
            x = x.to(device)
            t = t.to(device)
            y = net1(x)
            loss = criterion(y[0], t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')

shModel_trained = net1


: 

In [ ]:
from datetime import datetime
import pytz
def ShowTrainLoss(train_losses,val_losses,save):
    # 日本時間のタイムゾーンを取得
    jst = pytz.timezone('Asia/Tokyo')

    # 現在の日本時間を取得
    now = datetime.now(jst)

    # 時刻を指定された形式の文字列に変換
    time = "_"+now.strftime("%Y%m%d%H%M")

    # 学習曲線の可視化

    title = 'Training and Validation Losses Soft-hard'
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0,5)
    plt.legend()
    if save == True:
        plt.savefig("data/"+title+time+".pdf")

    plt.show()

ShowTrainLoss(train_losses,val_losses,False)

: 

In [ ]:

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y[0], dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            # x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net1(x)
            
            y_label = torch.argmax(y[0], dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)


: 

In [ ]:
# from models.dymn.model import get_model as get_dymn
# dymn10 = get_dymn(pretrained_name="dymn10_as")

# DataLoder(df,Dataset.matNum,25)
# FineTune(dymn10,5,1280,6)
# print(dymn10)
# TrainVall(dymn10,train_loader,val_loader,8)

# ShowTrainLoss(train_losses,val_losses,False)
# calc_acc(FineTune.net,val_loader)
# calc_acc(net,test_loader)

: 

### material 1

In [ ]:
# ラベルの指定
t_df = Dataset.matNum

# テンソル形式に変換
df_tenosor = torch.tensor(df, dtype=torch.float32)
df_tenosor = torch.unsqueeze(df_tenosor, 1)
label_tensor = torch.tensor(t_df, dtype=torch.int64)
print(df_tenosor.size())

# x と t を組み合わせて TensorDataset を作成
dataset = torch.utils.data.TensorDataset(df_tenosor, label_tensor)

# 各データセットのサンプル数を決定
# train : val: test = 50%　: 25% : 25%
n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# バッチサイズ
batch_size = 25

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

print(train_loader)

for batch in train_loader:
    print(batch)

: 

In [ ]:
matModel = shModel_trained

#パラメータの更新を許可
for param in matModel.parameters():
    param.requires_gred = True


# print(model.classifier)

# 出力層の最後だけ変更
# model.classifier[5] = torch.nn.Linear(1280,2)
matModel.classifier= nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=960, out_features=1280, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=527, bias=True),
    nn.Linear(in_features=527, out_features=176, bias=True),  # 新しい層
    nn.Linear(in_features=176, out_features=6, bias=True)  # 新しい層

)
print(matModel)

: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# エポックの数
max_epoch = 150
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# モデルのインスタンス化とデバイスへの転送
net2 = matModel.cuda()

criterion = F.cross_entropy
  
# 最適化手法の選択
optimizer = torch.optim.SGD(net2.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net2.train()
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net2(x)
        loss = criterion(y[0], t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net2.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
  
            x = x.to(device)
            t = t.to(device)
            y = net2(x)
            loss = criterion(y[0], t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')


matModel_trained = net2


: 

In [ ]:
from datetime import datetime
import pytz
def ShowTrainLoss(train_losses,val_losses,save):
    # 日本時間のタイムゾーンを取得
    jst = pytz.timezone('Asia/Tokyo')

    # 現在の日本時間を取得
    now = datetime.now(jst)

    # 時刻を指定された形式の文字列に変換
    time = "_"+now.strftime("%Y%m%d%H%M")

    # 学習曲線の可視化

    title = 'Training and Validation Losses Material'
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0,5)
    plt.legend()
    if save == True:
        plt.savefig("data/"+title+time+".pdf")

    plt.show()

ShowTrainLoss(train_losses,val_losses,True)

: 

In [ ]:

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y[0], dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            # x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net2(x)
            
            y_label = torch.argmax(y[0], dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)

: 

### object 1

In [ ]:
# ラベルの指定
t_df = Dataset.objectNum

# テンソル形式に変換
df_tenosor = torch.tensor(df, dtype=torch.float32)
df_tenosor = torch.unsqueeze(df_tenosor, 1)
label_tensor = torch.tensor(t_df, dtype=torch.int64)
print(df_tenosor.size())

# x と t を組み合わせて TensorDataset を作成
dataset = torch.utils.data.TensorDataset(df_tenosor, label_tensor)

# 各データセットのサンプル数を決定
# train : val: test = 50%　: 25% : 25%
n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# バッチサイズ
batch_size = 25

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

print(train_loader)

for batch in train_loader:
    print(batch)

: 

In [ ]:
objModel = matModel_trained

#パラメータの更新を許可
for param in objModel.parameters():
    param.requires_gred = True


# print(model.classifier)

# 出力層の最後だけ変更
# model.classifier[5] = torch.nn.Linear(1280,2)
objModel.classifier= nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=960, out_features=1280, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=527, bias=True),
    nn.Linear(in_features=527, out_features=176, bias=True),  # 新しい層
    nn.Linear(in_features=176, out_features=21, bias=True)  # 新しい層

)
print(objModel)

: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# エポックの数
max_epoch = 150
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# モデルのインスタンス化とデバイスへの転送
net3 = objModel.cuda()

criterion = F.cross_entropy
  
# 最適化手法の選択
optimizer = torch.optim.SGD(net3.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net3.train()
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net3(x)
        loss = criterion(y[0], t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net3.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
  
            x = x.to(device)
            t = t.to(device)
            y = net3(x)
            loss = criterion(y[0], t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')


objModel_trained = net3


: 

In [ ]:
from datetime import datetime
import pytz
def ShowTrainLoss(train_losses,val_losses,save):
    # 日本時間のタイムゾーンを取得
    jst = pytz.timezone('Asia/Tokyo')

    # 現在の日本時間を取得
    now = datetime.now(jst)

    # 時刻を指定された形式の文字列に変換
    time = "_"+now.strftime("%Y%m%d%H%M")

    # 学習曲線の可視化

    title = 'Training and Validation Losses Object'
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0,5)
    plt.legend()
    if save == True:
        plt.savefig("data/"+title+time+".pdf")

    plt.show()

ShowTrainLoss(train_losses,val_losses,True)

: 

In [ ]:

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y[0], dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            # x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net3(x)
            
            y_label = torch.argmax(y[0], dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)

: 

### soft-hard 2

In [ ]:
# ラベルの指定
t_df = Dataset.shNum

# テンソル形式に変換
df_tenosor = torch.tensor(df, dtype=torch.float32)
df_tenosor = torch.unsqueeze(df_tenosor, 1)
label_tensor = torch.tensor(t_df, dtype=torch.int64)
print(df_tenosor.size())

# x と t を組み合わせて TensorDataset を作成
dataset = torch.utils.data.TensorDataset(df_tenosor, label_tensor)

# 各データセットのサンプル数を決定
# train : val: test = 50%　: 25% : 25%
n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# バッチサイズ
batch_size = 25

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

print(train_loader)

for batch in train_loader:
    print(batch)

: 

In [ ]:
shModel_2 = objModel_trained

#パラメータの更新を許可
for param in shModel_2.parameters():
    param.requires_gred = False


# print(model.classifier)

# 出力層の最後だけ変更
# model.classifier[5] = torch.nn.Linear(1280,2)
shModel_2.classifier= nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=960, out_features=1280, bias=False),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features=527, bias=False),
    nn.Linear(in_features=527, out_features=176, bias=False),  # 新しい層
    nn.Linear(in_features=176, out_features=21, bias=False),  # 新しい層
    nn.ReLU(),
    nn.Linear(in_features=21, out_features=2, bias=True),  # 新しい層


)
print(shModel_2)

: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# エポックの数
max_epoch = 50
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# モデルのインスタンス化とデバイスへの転送
net4 = shModel_2.cuda()

criterion = F.cross_entropy
  
# 最適化手法の選択
optimizer = torch.optim.SGD(net4.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
for epoch in range(max_epoch):
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net4.train()
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net4(x)
        loss = criterion(y[0], t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net4.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
  
            x = x.to(device)
            t = t.to(device)
            y = net4(x)
            loss = criterion(y[0], t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')

shModel_2_trained = net4


: 

In [ ]:
from datetime import datetime
import pytz
def ShowTrainLoss(train_losses,val_losses,save):
    # 日本時間のタイムゾーンを取得
    jst = pytz.timezone('Asia/Tokyo')

    # 現在の日本時間を取得
    now = datetime.now(jst)

    # 時刻を指定された形式の文字列に変換
    time = "_"+now.strftime("%Y%m%d%H%M")

    # 学習曲線の可視化

    title = 'Training and Validation Losses Soft-hard'
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0,5)
    plt.legend()
    if save == True:
        plt.savefig("data/"+title+time+".pdf")

    plt.show()

ShowTrainLoss(train_losses,val_losses,False)

: 

In [ ]:

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y[0], dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            # x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net4(x)
            
            y_label = torch.argmax(y[0], dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)


: 

### material 2

In [ ]:
# ラベルの指定
t_df = Dataset.matNum

# テンソル形式に変換
df_tenosor = torch.tensor(df, dtype=torch.float32)
df_tenosor = torch.unsqueeze(df_tenosor, 1)
label_tensor = torch.tensor(t_df, dtype=torch.int64)
print(df_tenosor.size())

# x と t を組み合わせて TensorDataset を作成
dataset = torch.utils.data.TensorDataset(df_tenosor, label_tensor)

# 各データセットのサンプル数を決定
# train : val: test = 50%　: 25% : 25%
n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# バッチサイズ
batch_size = 25

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

print(train_loader)

for batch in train_loader:
    print(batch)

: 

In [ ]:
matModel_2 = shModel_2_trained

#パラメータの更新を許可
for param in matModel_2.parameters():
    param.requires_gred = False


# print(model.classifier)

# 出力層の最後だけ変更
# model.classifier[5] = torch.nn.Linear(1280,2)
matModel_2.classifier= nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=960, out_features=1280, bias=False),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features=527, bias=False),
    nn.Linear(in_features=527, out_features=176, bias=False),  # 新しい層
    nn.Linear(in_features=176, out_features=21, bias=False),  # 新しい層
    nn.ReLU(),
    nn.Linear(in_features=21, out_features=6, bias=True),  # 新しい層
    # nn.ReLU(),
    # nn.Linear(in_features=2, out_features=21, bias=True),  # 新しい層
    # nn.ReLU(),
    # nn.Linear(in_features=21, out_features=6, bias=True),  # 新しい層
)
print(matModel_2)

: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# エポックの数
epoch = 0
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# モデルのインスタンス化とデバイスへの転送
net5 = matModel_2.cuda()

criterion = F.cross_entropy
  
# 最適化手法の選択
optimizer = torch.optim.SGD(net5.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

# 学習ループ
# for epoch in range(max_epoch):
train_epoch_loss = 1.0
val_epoch_loss = 1.0

while train_epoch_loss > 0.1 or val_epoch_loss  > 0.1 or epoch<50:
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net5.train()
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net5(x)
        loss = criterion(y[0], t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net5.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
  
            x = x.to(device)
            t = t.to(device)
            y = net5(x)
            loss = criterion(y[0], t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')
    epoch+=1

matModel_2_trained = net5


: 

In [ ]:
from datetime import datetime
import pytz
def ShowTrainLoss(train_losses,val_losses,save):
    # 日本時間のタイムゾーンを取得
    jst = pytz.timezone('Asia/Tokyo')

    # 現在の日本時間を取得
    now = datetime.now(jst)

    # 時刻を指定された形式の文字列に変換
    time = "_"+now.strftime("%Y%m%d%H%M")

    # 学習曲線の可視化

    title = 'Training and Validation Losses Material'
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0,5)
    plt.legend()
    if save == True:
        plt.savefig("data/"+title+time+".pdf")

    plt.show()

ShowTrainLoss(train_losses,val_losses,True)

: 

In [ ]:

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y[0], dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            # x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net5(x)
            
            y_label = torch.argmax(y[0], dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)

: 

### object 2

In [ ]:
# ラベルの指定
t_df = Dataset.objectNum

# テンソル形式に変換
df_tenosor = torch.tensor(df, dtype=torch.float32)
df_tenosor = torch.unsqueeze(df_tenosor, 1)
label_tensor = torch.tensor(t_df, dtype=torch.int64)
print(df_tenosor.size())

# x と t を組み合わせて TensorDataset を作成
dataset = torch.utils.data.TensorDataset(df_tenosor, label_tensor)

# 各データセットのサンプル数を決定
# train : val: test = 50%　: 25% : 25%
n_train = int(len(dataset) * 0.5)
n_val = int(len(dataset) * 0.25)
n_test = len(dataset) - n_train - n_val
     
# それぞれのサンプル数を確認
print(n_train, n_val, n_test)

# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# バッチサイズ
batch_size = 25

      
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

print(train_loader)

for batch in train_loader:
    print(batch)

: 

In [ ]:
objModel_2 = matModel_2_trained

# パラメータの更新を許可
for param in objModel_2.parameters():
    param.requires_grad = False  # requires_gred -> requires_grad

# 新しい層の追加とBatch Normalizationの導入
objModel_2.classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=960, out_features=1280, bias=False),
    nn.BatchNorm1d(1280),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features=527, bias=False),
    nn.BatchNorm1d(527),
    nn.ReLU(),
    nn.Linear(in_features=527, out_features=176, bias=False),
    nn.BatchNorm1d(176),
    nn.ReLU(),
    nn.Linear(in_features=176, out_features=21, bias=False),
    nn.BatchNorm1d(21),
    nn.ReLU(),
    nn.Linear(in_features=21, out_features=2, bias=False),
    nn.BatchNorm1d(2),
    nn.ReLU(),
    nn.Linear(in_features=2, out_features=21, bias=False),
    nn.BatchNorm1d(21),
    nn.ReLU(),
    nn.Linear(in_features=21, out_features=6, bias=False),
    nn.BatchNorm1d(6),
    nn.ReLU(),
    nn.Linear(in_features=6, out_features=21, bias=True),
    nn.BatchNorm1d(21),
    nn.ReLU(),
    nn.Linear(in_features=21, out_features=21, bias=True),
)

print(objModel_2)


: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# エポックの数
epoch = 0
     
# モデルの初期化
torch.manual_seed(0)

# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# モデルのインスタンス化とデバイスへの転送
net6 = objModel_2.cuda()

criterion = F.cross_entropy
  
# 最適化手法の選択
optimizer = torch.optim.SGD(net6.parameters(), lr=0.1)
# エポックごとの訓練データの損失と検証データの損失を保存するリスト
train_losses = []
val_losses = []

train_epoch_loss = 1.0
val_epoch_loss = 1.0
# 学習ループ
while (train_epoch_loss > 0.1 or val_epoch_loss  > 0.2 )or epoch<100:
    train_epoch_loss = 0.0
    val_epoch_loss = 0.0
    
    # 訓練データでの学習
    net6.train()
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net6(x)
        loss = criterion(y[0], t)
        train_epoch_loss += loss.item()  # エポック全体の訓練データの損失に加算
        loss.backward()
        optimizer.step()
    
    # 訓練データでのエポックごとの平均損失を計算し保存
    train_epoch_loss /= len(train_loader)
    train_losses.append(train_epoch_loss)
    
    # 検証データでの評価
    net6.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, t = batch
  
            x = x.to(device)
            t = t.to(device)
            y = net6(x)
            loss = criterion(y[0], t)
            val_epoch_loss += loss.item()  # エポック全体の検証データの損失に加算
       
    # 検証データでのエポックごとの平均損失を計算し保存
    val_epoch_loss /= len(val_loader)
    val_losses.append(val_epoch_loss)
    
    # エポックごとに損失を表示
    print(f'Epoch [{epoch+1}/{max_epoch}], Train Loss: {train_epoch_loss:.4f}, Validation Loss: {val_epoch_loss:.4f}')
    epoch+=1

objModel_2_trained = net6


: 

In [ ]:
from datetime import datetime
import pytz
def ShowTrainLoss(train_losses,val_losses,save):
    # 日本時間のタイムゾーンを取得
    jst = pytz.timezone('Asia/Tokyo')

    # 現在の日本時間を取得
    now = datetime.now(jst)

    # 時刻を指定された形式の文字列に変換
    time = "_"+now.strftime("%Y%m%d%H%M")

    # 学習曲線の可視化

    title = 'Training and Validation Losses Material'
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0,5)
    plt.legend()
    if save == True:
        plt.savefig("data/"+title+time+".pdf")

    plt.show()

ShowTrainLoss(train_losses,val_losses,True)

: 

In [ ]:

# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y[0], dim=1)
# 予測値から最大となるクラスの番号を取り出した結果
y_label
# 目的変数
t
# 値が一致しているか確認
y_label == t
# int => float 
torch.sum(y_label == t) * 1.0
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc


# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            
            # x = torch.unsqueeze(x,1)
            
            x = x.to(device)
            t = t.to(device)
            y = net6(x)
            
            y_label = torch.argmax(y[0], dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc
      
# 検証データで確認
calc_acc(val_loader)
# テストデータで確認
calc_acc(test_loader)

: 

: 

: 